# Tarefa 03 

* Andreza Aparecida dos Santos - RA 164213 
* Gil Ribeiro de Carvalho - RA 225323 
* Thamiris Coelho - RA 187506

In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn import svm
from sklearn.cluster import DBSCAN

## Leitura dos Dados

Conjunto possui 900 dados "normais" e até 7 outliers.

In [66]:
data = pd.read_csv('dados3.csv')
data.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,-2.97,1.020,-2.340,3.460,1.630,0.157,-2.660,0.559,-5.27,1.960
1,4.30,-0.817,1.410,-2.160,0.673,0.870,-1.220,1.620,3.43,-0.771
2,-2.62,0.378,-1.010,1.430,-0.278,-0.384,0.613,-0.880,-2.14,0.465
3,2.38,-0.356,0.731,-1.250,0.391,0.362,-0.817,1.000,1.85,-0.260
4,1.87,-0.568,0.440,-0.856,0.401,0.576,-0.568,0.793,1.55,-0.412


In [67]:
data.shape

(907, 10)

In [68]:
# Variables

contamination = 0.01

Consideramos que existe uma contaminação de 1% nos dados, já que é um parâmetro configurável em alguns dos algorítmos implementados em Python. Isso resultaria em aproximadamente 9 outliers dentro do nosso conjunto de dados.

## Local Outlier Factor (por densidade)

In [89]:
neighbors = [2, 30, 100, 300]
iteration = 0

for neighbor_number in neighbors:
    clf = LocalOutlierFactor(n_neighbors=100, contamination=contamination)
    lof = clf.fit_predict(data)
    
    #lof_anomalies = np.where(lof == -1)[0]
    unique, counts = np.unique(lof, return_counts=True)
    lof_clusters[iteration] = dict(zip(unique, counts))
    print("Número de anomalias detectadas com {} vizinhos: {}".format(neighbor_number,lof_clusters[iteration][-1]))
    iteration += iteration

#clf = LocalOutlierFactor(n_neighbors=100, contamination=contamination)
#lof = clf.fit_predict(data)

#lof_anomalies = np.where(lof == -1)[0]
#unique, counts = np.unique(lof, return_counts=True)
#lof_clusters = dict(zip(unique, counts))

Número de anomalias detectadas com 2 vizinhos: 10
Número de anomalias detectadas com 30 vizinhos: 10
Número de anomalias detectadas com 100 vizinhos: 10
Número de anomalias detectadas com 300 vizinhos: 10


In [83]:
#print("Número de anomalias detectadas:", lof_clusters[-1])

Testando o LOF com número variado de vizinhos (2, 30, 100 e 300) e contaminação de 1%, obtivemos 10 outliers em qualquer caso, muito próximo dos 9 esperados, mas está fixo, podendo ser restringido pela contaminação.

## Isolation Forest

In [95]:
estimators = [30, 100, 400, 1000]
iteration = 0

for estimator in estimators:
    clf = IsolationForest(n_estimators=estimator, random_state=42, contamination=contamination).fit(data)
    isolation = clf.predict(data)
    
    #isolation_anomalies = np.where(isolation == -1)[0]
    unique, counts = np.unique(isolation, return_counts=True)
    isolation_clusters[iteration] = dict(zip(unique, counts))
    print("Número de anomalias detectadas com {} estimadores: {}".format(estimator,isolation_clusters[iteration][-1]))
    iteration += iteration


#clf = IsolationForest(random_state=42, contamination=contamination).fit(data)
#isolation = clf.predict(data)

#isolation_anomalies = np.where(isolation == -1)[0]
#unique, counts = np.unique(isolation, return_counts=True)
#isolation_clusters = dict(zip(unique, counts))

Número de anomalias detectadas com 30 estimadores: 10
Número de anomalias detectadas com 100 estimadores: 10
Número de anomalias detectadas com 400 estimadores: 10
Número de anomalias detectadas com 1000 estimadores: 10


In [96]:
#print("Número de anomalias detectadas:", isolation_clusters[-1])

Testando o Isolation Forest com número variado de estimadores (30, 100, 400 e 1000) e contaminação de 1%, obtivemos 10 outliers em qualquer caso, muito próximo dos 9 esperados, mas está fixo, podendo ser restringido pela contaminação.

## One Class SVM

In [105]:
nus = [0.001, 0.005, 0.01, 0.015, 0.02]
iteration = 0

for nu in nus:
    clf = svm.OneClassSVM(nu=nu, kernel="rbf").fit(data)
    ol = clf.predict(data)

    #ol_anomalies = np.where(ol == -1)[0]
    unique, counts = np.unique(ol, return_counts=True)
    ol_clusters[iteration] = dict(zip(unique, counts))
    print("Número de anomalias detectadas com nu de {}: {}".format(nu,ol_clusters[iteration][-1]))
    iteration += iteration
       
#clf = svm.OneClassSVM(nu=contamination, kernel="rbf").fit(data)
#ol = clf.predict(data)

#ol_anomalies = np.where(ol == -1)[0]
#unique, counts = np.unique(ol, return_counts=True)
#ol_clusters = dict(zip(unique, counts))

Número de anomalias detectadas com nu de 0.001: 9
Número de anomalias detectadas com nu de 0.005: 6
Número de anomalias detectadas com nu de 0.01: 7
Número de anomalias detectadas com nu de 0.015: 14
Número de anomalias detectadas com nu de 0.02: 19


In [98]:
#print("Número de anomalias detectadas:", ol_clusters[-1])

Testando o One Class SVM com diferentes nu (0.001, 0.005, 0.01, 0.015 e 0.02), encontramos variados valores de outliers, observado uma queda inicial entre 0.001 e 0.005, voltando a subir já com 0.01. Constatamos que após o valor de nu de 0.01, já no valor de 0.015, o número de outliers detectado dispara, passando do valor da contaminação esperada. Consideraremos então o valor de nu de X que proprociona Y outliers, estando dentro dos valores esperados de contaminação.

## DBSCan (por densidade)

In [108]:
eps_list = [0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5]
samples_list = [2, 4, 8, 16]
iteration = 0

for samples in samples_list:
    print("Utilizando {} amostras vizinhas:".format(samples))
    for eps in eps_list:     
        outlier_detection = DBSCAN(min_samples = samples, eps = eps)
        clusters = outlier_detection.fit_predict(data)

        #dbscan_anomalies = np.where(clusters == -1)[0]
        unique, counts = np.unique(clusters, return_counts=True)
        dbscan_clusters[iteration] = dict(zip(unique, counts))
        print("Número de anomalias detectadas com eps de {}: {}".format(eps, dbscan_clusters[iteration][-1]))
        iteration += iteration


#outlier_detection = DBSCAN(min_samples = 2, eps = 3)
#clusters = outlier_detection.fit_predict(data)

#dbscan_anomalies = np.where(clusters == -1)[0]
#unique, counts = np.unique(clusters, return_counts=True)
#dbscan_clusters = dict(zip(unique, counts))

Utilizando 2 amostras vizinhas:
Número de anomalias detectadas com eps de 0.5: 15
Número de anomalias detectadas com eps de 1: 7
Número de anomalias detectadas com eps de 1.5: 7
Número de anomalias detectadas com eps de 2: 7
Número de anomalias detectadas com eps de 2.5: 7
Número de anomalias detectadas com eps de 3: 7
Número de anomalias detectadas com eps de 3.5: 5
Número de anomalias detectadas com eps de 4: 4
Número de anomalias detectadas com eps de 4.5: 3
Número de anomalias detectadas com eps de 5: 1
Utilizando 4 amostras vizinhas:
Número de anomalias detectadas com eps de 0.5: 17
Número de anomalias detectadas com eps de 1: 7
Número de anomalias detectadas com eps de 1.5: 7
Número de anomalias detectadas com eps de 2: 7
Número de anomalias detectadas com eps de 2.5: 7
Número de anomalias detectadas com eps de 3: 7
Número de anomalias detectadas com eps de 3.5: 5
Número de anomalias detectadas com eps de 4: 4
Número de anomalias detectadas com eps de 4.5: 3
Número de anomalias d

In [101]:
#print("Número de anomalias detectadas:", dbscan_clusters[-1])

Testando o DBScan com variações de amostras vizinhas (2, 4, 8 e 16) e eps (0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5 e 5), constatamos que para os variados números de amostras vizinhas, dentro da faixa de eps de 1 ou 1.5 a 3, existe uma constância no valor de outliers detectados, no caso, 7, próximo dos 9 esperados com a contaminação de 1%. Portanto consideraremos o valor de 4 amostras vizinhas e eps de 2, ficando com 7 outliers, pouco abaixo da contaminação esperada.

## Combinando os resultados

In [77]:
total_anomalies = list(set(dbscan_anomalies) & set(ol_anomalies) & set(isolation_anomalies) & set(lof_anomalies))

In [78]:
print("Número de anomalias combinando os resultados:", len(total_anomalies))

Número de anomalias combinando os resultados: 4


In [79]:
print("Dados anomalos encontrados combinando os resultados de todas as técnicas aplicadas")
data.loc[data.index.isin(total_anomalies)]

Dados anomalos encontrados combinando os resultados de todas as técnicas aplicadas


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
0,-2.97,1.020,-2.340,3.46,1.630,0.157,-2.660,0.559,-5.270,1.96
358,-1.06,-0.771,0.273,-1.49,-1.930,-0.709,3.400,-1.430,2.410,-1.02
554,-3.68,-1.890,-4.730,1.19,0.696,0.306,-0.464,1.630,-0.680,1.66
832,-2.15,0.469,-1.350,-1.12,-1.200,-2.070,0.909,0.224,0.527,1.67
